In [38]:
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter
import numpy as np
import os, json, fasttext, re
from lxml import etree as ET
import copy
import unicodedata as ud

In [2]:
# Enable tqdm for pandas
tqdm.pandas()

# Create a flatten function
flatten = lambda t: [item for sublist in t for item in sublist]

/Users/martijn/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Initial data cleaning

In [3]:
data_root = 'data/repo-genmymodel-uml/data'

In [4]:
filenames = next(os.walk(data_root), (None, None, []))[2]

# Start wrangling

In [5]:
len(filenames)

352216

In [6]:
filenames[0]

'_cDm6cGfBEeepGY91r9Nuow.xmi.uml'

## Defining patterns

### XML UML tag patterns

In ed9c3064-7fb5-4ec0-bded-c114251076fa.xmi.uml, packagedElement tags are used to define classes (```uml:Class```): 

```
<packagedElement xsi:type="uml:Class" xmi:id="_eI6xoFdMEDCGTb6ZuLm3Mw" name="Client">
```

The ownedAttribute tag holds attribute names:

```
<ownedAttribute xmi:id="_GVIe2JUXEeqqGZh46IEtXQ" name="id">
```

## Search for patterns in files


In [17]:
def get_classes_and_attributes_from_uml(filepath):
    root = ET.parse(data_root + '/' + filepath)
    
    # Gather classes and attributes using defined patterns
    classes = root.xpath("//packagedElement[@xsi:type='uml:Class']/@name", namespaces={'xsi': 'http://www.w3.org/2001/XMLSchema-instance'})
    attributes = root.xpath("//packagedElement[@xsi:type='uml:Class']/ownedAttribute/@name", namespaces={'xsi': 'http://www.w3.org/2001/XMLSchema-instance'})
    
    # Save classes and attributes in list if they hold values
    found_data = {}
    if len(classes) > 0:
        found_classes = [x for x in [x.strip() for x in classes] if x != '']
        if len(found_classes) > 0:
            found_data['classes'] = found_classes
    if len(attributes) > 0:
        found_attributes = [x for x in [x.strip() for x in attributes] if x != '']
        if len(found_classes) > 0:
            found_data['attributes'] = found_attributes
    
    return found_data

In [8]:
get_classes_and_attributes_from_uml('ed9c3064-7fb5-4ec0-bded-c114251076fa.xmi.uml')

{'classes': ['Client', 'Adaptor', 'Adaptee']}

In [9]:
data_dict = {}

for file in tqdm(filenames):
    try:
        data = get_classes_and_attributes_from_uml(file)
    except Exception as e:
        print(e)
    
    if len(data.keys()) > 0:
        data_dict[file] = data

Start tag expected, '<' not found, line 1, column 1 (_mu7qEH1yEeShroDxTnL5_A.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_m_Rk4KwTEea217yLRs2OoA.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_QccQIO_IEeSujetaMhozjg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (2f9d0cc3-7598-4e0e-a1ce-5aa700687522.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_5yFvIKYsEei1evnFuWTWfg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_TPfGIDM0EeSm5tDnu9wFCA.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_9y-TQJ4sEeiGsdp6eB6oaw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_y2OWACeREeShcfbUiDP6ug.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_9w0aYK1CEDGNWLI2ml8Hng.xmi.uml, line 1)
Opening and ending tag mismatch: link line 1 and head, line 1, column 259 (_jOJ8gOS7EeekKLRLyKXO4Q.xmi.uml, line 1)
Opening and ending tag mi

Start tag expected, '<' not found, line 1, column 1 (_ofzqkLpaEeiBfp11hItRuw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (54749843-b387-4ee8-8f4b-c2630321f1af.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_uW1j8CUfEDGlaJTZcof-Dw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_g24SgCtMEDGWq_RLO4Gl4w.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_YtikoEm5EeS_velUIC8tvA.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (6e20cd3f-25ed-44d9-b339-5ba8a2e148bb.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (f12c8f43-8bad-4035-9fb0-3a77f60d0472.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (5d02602f-7397-4b4d-a50d-f7302baab3f0.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_5x-WEJzsEeWWRMvT9FgLDg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (031bbef7-da12-4a68-a65d-90fe85b86bc7.xmi.uml, lin

Start tag expected, '<' not found, line 1, column 1 (cb20410d-6ab4-4cdb-b95b-d9831155310e.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_tb5msO-UEeSujetaMhozjg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_cKV-kDzHEeSn3ZXpVIPWYw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_jbCAEAquEeWNVPA9hBwttA.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_D9v3ELwQEeSvIovPojnUsw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (1648bfb8-15cc-4d99-be61-9bbf29974caf.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (f2aa59dc-3aeb-4787-aaee-1a28414022bd.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_DLNxgOx6EeW6A_AbwsZiiQ.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_rZiSsDNdEeSm5tDnu9wFCA.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_dUA9wNHzEeSyKtHqJeGVMw.xmi.uml, line 1)
Start tag expected, '

Start tag expected, '<' not found, line 1, column 1 (_Jk4CYL6nEeSBntdMhqoN9Q.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_jBy1oIhuEeWSIKoWblTvZg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (__SM48KzwEeS57ICseEaz5A.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_aXCtoPgDEeSJkrdQR2VoEQ.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (5cf8aafe-fb28-4cd7-ad7f-99d42f938248.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_pAJ10J44EeWWRMvT9FgLDg.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (4223130b-d098-41d8-9acf-f5ef087d71cf.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_6CcSAFO4EeS7SK6lkmJgjw.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (_fxlTwNlDEeSpEswBeJxMRQ.xmi.uml, line 1)
Start tag expected, '<' not found, line 1, column 1 (639942fa-1753-433a-acf2-c189766be951.xmi.uml, line 1)
Start tag expected, '

In [10]:
data_dict

{'5fae885c-3fde-430f-a3c5-ff44b81f2668.xmi.uml': {'classes': ['ClassA',
   'ClassB',
   'ClassC',
   'ClassD',
   'ClassE',
   'ClassF',
   'ClassG',
   'ClassH',
   'ClassI']},
 '_oVCEUIHiEeeveJPbhFhy-g.xmi.uml': {'classes': ['SitePage', 'SiteSection'],
  'attributes': ['id',
   'status',
   'name',
   'url',
   'id',
   'locked',
   'name',
   'defaultPage']},
 '20b51f77-a952-4206-b35c-b47acd0c631f.xmi.uml': {'classes': ['MoteurImpl',
   'Buffer',
   'Selection',
   'PressePapier'],
  'attributes': ['texte', 'sel', 'attribute']},
 'e8a7e84d-73b3-40d2-ac78-f550566b4de8.xmi.uml': {'classes': ['Client',
   'AdapterImpl',
   'Adaptee',
   'Context',
   'StateA',
   'StateB',
   'Singleton',
   'Abstraction',
   'RefinedAbstraction',
   'ConcreteImplementor',
   'CreatorImpl',
   'Product',
   'ConcreteObserver',
   'ConcreteSubject'],
  'attributes': ['instance']},
 '_oWwJoHfLEeqeQcxm9hmzHw.xmi.uml': {'classes': ['ShoppingCart',
   'Order',
   'LineItem',
   'Account',
   'Customer'],
  

In [11]:
for file in data_dict.keys():
    if 'classes' not in data_dict[file].keys() or len(data_dict[file]['classes']) == 0:
        print(file)
        print(data_dict[file])

In [67]:
# Get total amount of unique classes
print(len(np.unique(np.array(list(map(lambda x: x.strip(), flatten([data_dict[file]['classes'] for file in data_dict.keys()])))))))

# Get total amount of unique attributes
print(len(np.unique(np.array(flatten([[str(attr).strip() for attr in values['attributes']] for values in data_dict.values() if 'attributes' in values.keys()])))))

344981
455730


In [ ]:
with open('data/genmymodel_uml_extracted_metadata.json', 'w') as fp:
    json.dump(data_dict, fp)

## First "raw" data cleaning cycle

In [92]:
with open('data/genmymodel_uml_extracted_metadata.json') as json_file:
    data_dict = json.load(json_file)

In [93]:
def should_be_kept(text):
    # Check if larger than 1 character
    if len(text) > 1:
        # Remove function calls '_8Ux_QFaxEeSu48YT12vqpQ.xmi.uml'
        if text[-2:] == '()':
            return False

        # Remove getters and setters '_UWr2cM_JEeeLcIicqHdTUQ.xmi.uml' '_va9xEFzhEeqK2M3E1LfZ7Q.xmi.uml'
        if re.search(r'^[gs]et(?:[A-Z]|_)\w+', text):
            return False

        # Remove dot seperated widgets/views and filenames '_tGyV0EbLEeeTJ_4Vl2J2rQ.xmi.uml' '_WZQeEMJ8EeSII650IQ0Z1w.xmi.uml'
        if re.search(r'^\w+\.\w+(?:\.\w+)*', text):
            return False

        # Remove comma-separated attributes 'f6e1ef7a-107f-419e-ab7d-a51aa49c13d2.xmi.uml'
        if re.search(r'^\w+?(?:,\w+?)+', text):
            return False
    else: 
        return False
    
    # All cases do not match, text may stay
    return True

data_dict = {
    filename: {
        metadata: list(filter(lambda x: should_be_kept(x), texts))
        for metadata, texts in extracted_data.items()
    }
    for filename, extracted_data in tqdm(data_dict.items())
}

In [70]:
# Get total amount of unique classes
print(len(np.unique(np.array(list(map(lambda x: x.strip(), flatten([data_dict[file]['classes'] for file in data_dict.keys()])))))))

# Get total amount of unique attributes
print(len(np.unique(np.array(flatten([[str(attr).strip() for attr in values['attributes']] for values in data_dict.values() if 'attributes' in values.keys()])))))

336395
447945


In [94]:
# Remove classes/attributes keys if they don't hold any values
iteration_copy = copy.deepcopy(data_dict)
for file in tqdm(iteration_copy.keys()):
    for metadata in iteration_copy[file].keys():
        if len(iteration_copy[file][metadata]) == 0:
            del data_dict[file][metadata]

# Remove files from dataset if they don't hold any classes and attributes anymore
iteration_copy = copy.deepcopy(data_dict)
for file in tqdm(iteration_copy.keys()):
    if len(iteration_copy[file].keys()) == 0:
        del data_dict[file]

# Clear up memory
del iteration_copy

In [95]:
with open('data/genmymodel_uml_extracted_metadata_cleaned1.json', 'w') as fp:
    json.dump(data_dict, fp)

In [96]:
data_dict

{'5fae885c-3fde-430f-a3c5-ff44b81f2668.xmi.uml': {'classes': ['ClassA',
   'ClassB',
   'ClassC',
   'ClassD',
   'ClassE',
   'ClassF',
   'ClassG',
   'ClassH',
   'ClassI']},
 '_oVCEUIHiEeeveJPbhFhy-g.xmi.uml': {'classes': ['SitePage', 'SiteSection'],
  'attributes': ['id',
   'status',
   'name',
   'url',
   'id',
   'locked',
   'name',
   'defaultPage']},
 '20b51f77-a952-4206-b35c-b47acd0c631f.xmi.uml': {'classes': ['MoteurImpl',
   'Buffer',
   'Selection',
   'PressePapier'],
  'attributes': ['texte', 'sel', 'attribute']},
 'e8a7e84d-73b3-40d2-ac78-f550566b4de8.xmi.uml': {'classes': ['Client',
   'AdapterImpl',
   'Adaptee',
   'Context',
   'StateA',
   'StateB',
   'Singleton',
   'Abstraction',
   'RefinedAbstraction',
   'ConcreteImplementor',
   'CreatorImpl',
   'Product',
   'ConcreteObserver',
   'ConcreteSubject'],
  'attributes': ['instance']},
 '_oWwJoHfLEeqeQcxm9hmzHw.xmi.uml': {'classes': ['ShoppingCart',
   'Order',
   'LineItem',
   'Account',
   'Customer'],
  

## Language detection

Because the dataset contains attributes and classes in lots of languages, we want to identify all the English files that contain attributes and classes, and filter out all the rest. For this, we use FastText.

In [99]:
# Reload data_dict from file
with open('data/genmymodel_uml_extracted_metadata_cleaned1.json') as json_file:
    data_dict = json.load(json_file)

    # Make sure all programmic cases are turned into spaces like "normal" text for better language detection
    cleaned_data = {
        file: {
            key: list(map(lambda x: ' '.join(list(map(lambda data_str: re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', data_str.replace('_', ' ').strip()).lower(), x.split(' ')))), value))
            for key, value in metadata.items()
        } 
        for file, metadata in tqdm(data_dict.items())
    }

In [100]:
cleaned_data

{'5fae885c-3fde-430f-a3c5-ff44b81f2668.xmi.uml': {'classes': ['class a',
   'class b',
   'class c',
   'class d',
   'class e',
   'class f',
   'class g',
   'class h',
   'class i']},
 '_oVCEUIHiEeeveJPbhFhy-g.xmi.uml': {'classes': ['site page', 'site section'],
  'attributes': ['id',
   'status',
   'name',
   'url',
   'id',
   'locked',
   'name',
   'default page']},
 '20b51f77-a952-4206-b35c-b47acd0c631f.xmi.uml': {'classes': ['moteur impl',
   'buffer',
   'selection',
   'presse papier'],
  'attributes': ['texte', 'sel', 'attribute']},
 'e8a7e84d-73b3-40d2-ac78-f550566b4de8.xmi.uml': {'classes': ['client',
   'adapter impl',
   'adaptee',
   'context',
   'state a',
   'state b',
   'singleton',
   'abstraction',
   'refined abstraction',
   'concrete implementor',
   'creator impl',
   'product',
   'concrete observer',
   'concrete subject'],
  'attributes': ['instance']},
 '_oWwJoHfLEeqeQcxm9hmzHw.xmi.uml': {'classes': ['shopping cart',
   'order',
   'line item',
   'acco

In [59]:
PRETRAINED_MODEL_PATH = 'data/lid.176.bin'

model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [102]:
# Approximate the language from the extracted classes and attributes
def get_file_language(metadata):
    # Combine all classes and attributes into one array
    all_metadata = sum(metadata.values(), [])
    
    # Detect language for each word
    predictions = model.predict(all_metadata)
    
    # Specific approach for longer texts due to issue with habit of using English in programming
    def get_field_level_prediction(string):
        # Split into single words to detect on word level
        words = string.split(' ')
        
        # Get single-word predictions
        string_predictions = np.array(model.predict(words))
        
        # Return the appropriate language
        if len(string_predictions) == 1:
            return string_predictions[0][0][0]
        else:
            # Option 1: all identical language identification
            if len(np.unique(string_predictions[0])) == 1:
                return string_predictions[0][0][0]
            
            # Option 2: combination of an English programmatic word with another language, e.g. "hovedmenu view"
            elif len(string_predictions) == 2 and any('__label__en' in label for label in string_predictions):
                return [x for x in map(lambda x: x[0], string_predictions[0]) if x != '__label__en'][0]
            
            # Option 3: majority vote
            else:
                return Counter(map(lambda x: x[0], string_predictions[0])).most_common(1)[0][0]
        
    # Apply to all fields, identify language for all classes and attributes
    predictions = list(map(lambda x: get_field_level_prediction(x), all_metadata))

    # Get the most common language from all attributes
    return Counter(predictions).most_common(1)[0][0]

In [103]:
labeled_cleaned_data = {
    key: {**value, **{'lang': get_file_language(value)}}
    for key, value in tqdm(cleaned_data.items())
}

In [104]:
labeled_cleaned_data

{'5fae885c-3fde-430f-a3c5-ff44b81f2668.xmi.uml': {'classes': ['class a',
   'class b',
   'class c',
   'class d',
   'class e',
   'class f',
   'class g',
   'class h',
   'class i'],
  'lang': '__label__en'},
 '_oVCEUIHiEeeveJPbhFhy-g.xmi.uml': {'classes': ['site page', 'site section'],
  'attributes': ['id',
   'status',
   'name',
   'url',
   'id',
   'locked',
   'name',
   'default page'],
  'lang': '__label__en'},
 '20b51f77-a952-4206-b35c-b47acd0c631f.xmi.uml': {'classes': ['moteur impl',
   'buffer',
   'selection',
   'presse papier'],
  'attributes': ['texte', 'sel', 'attribute'],
  'lang': '__label__fr'},
 'e8a7e84d-73b3-40d2-ac78-f550566b4de8.xmi.uml': {'classes': ['client',
   'adapter impl',
   'adaptee',
   'context',
   'state a',
   'state b',
   'singleton',
   'abstraction',
   'refined abstraction',
   'concrete implementor',
   'creator impl',
   'product',
   'concrete observer',
   'concrete subject'],
  'attributes': ['instance'],
  'lang': '__label__en'},
 '

In [105]:
with open('data/genmymodel_uml_extracted_metadata_annotated.json', 'w') as fp:
    json.dump(labeled_cleaned_data, fp)

# Further data cleaning

In [155]:
with open('data/genmymodel_uml_extracted_metadata_annotated.json') as json_file:
    labeled_cleaned_data = json.load(json_file)

In [156]:
filtered_files = {key: value for key, value in labeled_cleaned_data.items() if value['lang'] == '__label__en'}

In [157]:
# Remove hand-picked files
for file in ['_0se6kFNjEeS7SK6lkmJgjw.xmi.uml', '_zeUN8FJjEeeTnI9B59buBQ.xmi.uml', '_HKUJMMGAEeSBntdMhqoN9Q.xmi.uml', '48ecca9d-91d4-44f2-a21f-b0285228d504.xmi.uml']:
    del filtered_files[file]

In [158]:
filtered_files['_vZBssKMfEeScLuNN63kNaQ.xmi.uml']

{'classes': ['issue',
  'issue comparator',
  'git issues',
  'issue list',
  'issues manager',
  'parser',
  'parser copy',
  'pre- commit- validation- hook'],
 'attributes': ['id', 'assignee', 'milestone', 'etc'],
 'lang': '__label__en'}

In [159]:
def text_transform(text):
    # Remove HTML tags and other tags '_WZQeEMJ8EeSII650IQ0Z1w.xmi.uml'
    text = re.sub(r'<\w+?>', '', text)

    # Remove dollar sign at the beginning '_2p9bEMasEeadwOIOqK-0Fw.xmi.uml'
    text = re.sub(r'^\$(?=\w+)', '', text)
    
    # Remove all "my" things '_8y0sAIKDEeeveJPbhFhy-g.xmi.uml'
    text = text.replace('my ', '')

    # Replace all parentheses with spaces '_2A0eUK99EealN5YbbDMqoA.xmi.uml' '_UxXH4Ji_EeWP9KF1Y_wKUg.xmi.uml'
    text = re.sub(r'\(\w+?\)', '', text)
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    
    # Remove all attached numbers '_cOog0CIgEeisAYMSV00L2Q.xmi.uml'
    text = re.sub(r'(?<=\w)\d', '', text)

    # Replace "impl" with something '19515821-de9c-4371-ab13-0c10c1f0741e.xmi.uml'
    text = re.sub(r'\bimpl\b', 'implementation', text)

    # Remove questions '_arOCQOkeEeiV94kHgjpOMg.xmi.uml'
    text = re.sub(r'(?<=\w)\?', '', text)

    # Re-combine dashes '_vZBssKMfEeScLuNN63kNaQ.xmi.uml'
    text = re.sub(r'-\s(?=\w+?)', '-', text)

    # Turn ref abbreviation into reference '_w-4dMCDQEeqqcaoAsxFIeg.xmi.uml'
    text = re.sub(r'\bref\b', 'reference', text)

    # Remove single letter with space at beginning '_zTVWYHphEeas_KSqa81bqw.xmi.uml'
    text = re.sub(r'^\w\s(?=\w+)', '', text)
    
    # Remove colons
    text = text.replace(':', ' ')

    # Remove duplicate spaces
    text = re.sub(r'\s{2,}', ' ', text)
    
    # Remove scripted pattern " comments";
    text = re.sub(r'" (.+)";{0,1}', r'\1', text)
    
    # Remove all blokhaken
    text = re.sub(r'\[.*\]', '', text)
    
    # Remove dash to start
    text = re.sub(r'^-', '', text)
    
    # Remove plus to start
    text = re.sub(r'^\+', '', text)
    
    # Replace # with number when at end
    text = re.sub(r' ?#$', ' number', text)
    
    # Replace # with number at start
    text = re.sub(r'^#', 'number ', text)
    
    # Replace all dashes with spaces
    text = text.replace('-', ' ')
    
    # Remove * + 
    text = re.sub('\*$', '', text)
    text = re.sub('[\?]', '', text)
    
    # Remove tags
    text = re.sub(r'.+?< ([\w\s]+)>', r'\1', text)
    text = re.sub(r'^<>', '', text)
    text = re.sub(r'<$', '', text)
    
    # Change ampersand to "and"
    text = text.replace(' & ', ' and ')
    
    # Replace backslash in front
    text = re.sub(r'^\\ ', '', text)
    
    # Remove duplicate spaces
    text = text.replace('  ', ' ')
    
    # Trim output
    return text.strip()

text_transform('abstract resource item< style sheet item, style sheet params,url>')

'abstract resource item< style sheet item, style sheet params,url>'

In [160]:
filtered_files = {
    filename: {
        metadata: list(map(lambda x: text_transform(x), texts))
        for metadata, texts in extracted_data.items()
        if metadata != 'lang'
    }
    for filename, extracted_data in tqdm(filtered_files.items())
}

In [161]:
filtered_files

{'5fae885c-3fde-430f-a3c5-ff44b81f2668.xmi.uml': {'classes': ['class a',
   'class b',
   'class c',
   'class d',
   'class e',
   'class f',
   'class g',
   'class h',
   'class i']},
 '_oVCEUIHiEeeveJPbhFhy-g.xmi.uml': {'classes': ['site page', 'site section'],
  'attributes': ['id',
   'status',
   'name',
   'url',
   'id',
   'locked',
   'name',
   'default page']},
 'e8a7e84d-73b3-40d2-ac78-f550566b4de8.xmi.uml': {'classes': ['client',
   'adapter implementation',
   'adaptee',
   'context',
   'state a',
   'state b',
   'singleton',
   'abstraction',
   'refined abstraction',
   'concrete implementor',
   'creator implementation',
   'product',
   'concrete observer',
   'concrete subject'],
  'attributes': ['instance']},
 '_oWwJoHfLEeqeQcxm9hmzHw.xmi.uml': {'classes': ['shopping cart',
   'order',
   'line item',
   'account',
   'customer'],
  'attributes': ['creation date', 'id', 'quantity', 'price', 'id']},
 '_OR8QgFq_EeeTF9GWmq2Xvg.xmi.uml': {'classes': ['exam',
   'que

In [162]:
latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha())

def not_three_consecutive_chars(unistr):
    start = unistr[0]
    
    for char in unistr[1:]:
        if char == start[0]:
            start = start + char
            
            if len(start) > 2:
                return False
        else:
            start = char
    
    return True

In [163]:
# Create non-changing copy for iteration through the files
iteration_copy = copy.deepcopy(filtered_files)

for file in iteration_copy.keys():
    for metadata in iteration_copy[file].keys():
        # Remove all empty strings and non-alphanumerical entries
        new_metadata = list(filter(lambda x: x != '' and len(x) > 1 and x.replace(' ', '').replace("'", '').isalnum() and only_roman_chars(x) and any(c.isalpha() for c in x) and len(set(filter(str.isalpha, x))) != 1 and not re.match(r'(?:aa)|(?:uu)|(?:yy)|(?:ii)|(?:jj)', x) and not_three_consecutive_chars(x) and not '0f' in x and model.predict([x])[0][0][0] == '__label__en', filtered_files[file][metadata]))
        
        # Either remove the now-empty metadata or change it in the file
        if len(new_metadata) > 0:
            filtered_files[file][metadata] = new_metadata
        else:
            del filtered_files[file][metadata]
    
    # Remove file if it doesn't contain metadata anymore after filtering
    if len(filtered_files[file].keys()) == 0:
        del filtered_files[file]

In [164]:
with open('data/genmymodel_uml_extracted_metadata_final.json', 'w') as fp:
    json.dump(filtered_files, fp)